In [86]:
from pprint import pprint
import pandas as pd
import time
import IPython
pd.options.mode.chained_assignment = None  # default='warn'

# Opportunities

In [31]:
df = pd.DataFrame(pd.read_csv("../logs/arbbot.csv"))
df["humanTimestamp"] = pd.to_datetime(df['blockTimestamp'], unit='s')
df["runTime"] = df["endTimestamp"] - df["startTimestamp"]

In [32]:
# Limit to hour span
hour_limit = 24
sec_limit = hour_limit * 3600
df = df[df.blockTimestamp > time.time()-sec_limit]

In [33]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
under10 = df[(df.netProfit>0)&(df.optimalAmount<10)].sort_values(by="netProfit", ascending=0)
print(f"Amount of opp with <10eth input (24h): {len(under10)}")
print(f"Average input amount (24h): {under10.optimalAmount.mean()} eth")
print(f"Average net profit (24h): {under10.netProfit.mean()} eth")

Amount of opp with <10eth input (24h): 9
Average input amount (24h): 3.7957777777777775 eth
Average net profit (24h): 0.007238406111111113 eth


In [34]:
# df[df["netProfit"]>0].sort_values(by="netProfit", ascending=0)
above10 = df[(df.netProfit>0)&(df.optimalAmount>10)].sort_values(by="netProfit", ascending=0)
print(f"Amount of opp (24h): {len(above10)}")
print(f"Average input amount (24h): {above10.optimalAmount.mean()} eth")
print(f"Average net profit (24h): {above10.netProfit.mean()} eth")

Amount of opp (24h): 7
Average input amount (24h): 69.15285714285713 eth
Average net profit (24h): 0.12449828571428574 eth


In [99]:
df[(df.blockNumber<11596552)&(df.instrName=="weth2snx2weth_sushiswap2Uniswap")].sort_values(by="blockNumber", ascending=0)

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp,humanTimestamp,runTime
79432,11596550.0,1609876317,weth2snx2weth_sushiswap2Uniswap,I0014,14.34,0.04821,0.006209,0.042,240000,1.609876e+09,1.609876e+09,2021-01-05 19:51:57,1.974260
79429,11596549.0,1609876295,weth2snx2weth_sushiswap2Uniswap,I0014,14.34,0.04821,0.006209,0.042,240000,1.609876e+09,1.609876e+09,2021-01-05 19:51:35,1.639474


In [70]:
df[(df.blockNumber<11598808)&(df.instrName=="weth2band2weth_sushiswap2Uniswap")].sort_values(by="blockNumber", ascending=0)

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp,humanTimestamp,runTime
83392,11598807.0,1609906954,weth2band2weth_sushiswap2Uniswap,I0016,2.780,0.021650,-0.003072,0.02472,240000,1.609907e+09,1.609907e+09,2021-01-06 04:22:34,1.631059
83389,11598806.0,1609906934,weth2band2weth_sushiswap2Uniswap,I0016,2.780,0.021650,-0.003065,0.02471,240000,1.609907e+09,1.609907e+09,2021-01-06 04:22:14,2.098821
83386,11598805.0,1609906930,weth2band2weth_sushiswap2Uniswap,I0016,2.780,0.021650,-0.003065,0.02471,240000,1.609907e+09,1.609907e+09,2021-01-06 04:22:10,2.105093
83382,11598804.0,1609906924,weth2band2weth_sushiswap2Uniswap,I0016,2.181,0.013270,-0.011450,0.02472,240000,1.609907e+09,1.609907e+09,2021-01-06 04:22:04,1.963472
83379,11598803.0,1609906902,weth2band2weth_sushiswap2Uniswap,I0016,2.181,0.013270,-0.016730,0.03000,240000,1.609907e+09,1.609907e+09,2021-01-06 04:21:42,2.585154
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79006,11596320.0,1609873432,weth2band2weth_sushiswap2Uniswap,I0016,0.627,0.001085,-0.025080,0.02616,240000,1.609873e+09,1.609873e+09,2021-01-05 19:03:52,2.067933
79004,11596319.0,1609873426,weth2band2weth_sushiswap2Uniswap,I0016,0.627,0.001085,-0.025080,0.02616,240000,1.609873e+09,1.609873e+09,2021-01-05 19:03:46,1.389085
79002,11596318.0,1609873422,weth2band2weth_sushiswap2Uniswap,I0016,0.627,0.001085,-0.020330,0.02142,240000,1.609873e+09,1.609873e+09,2021-01-05 19:03:42,1.942810
79000,11596317.0,1609873419,weth2band2weth_sushiswap2Uniswap,I0016,0.627,0.001085,-0.024840,0.02592,240000,1.609873e+09,1.609873e+09,2021-01-05 19:03:39,1.969396


## Best opportunities so far

In [45]:
df.sort_values(by="netProfit", ascending=0).head(5)

,blockNumber,blockTimestamp,instrName,instrId,optimalAmount,grossProfit,netProfit,gasCost,gasAmount,startTimestamp,endTimestamp,humanTimestamp,runTime
83581,11598878.0,1609907898,weth2dai2weth_sushiswap2Uniswap,I0006,141.400,0.44360,0.40810,0.03552,240000,1.609908e+09,1.609908e+09,2021-01-06 04:38:18,2.243185
83577,11598877.0,1609907868,weth2dai2weth_sushiswap2Uniswap,I0006,141.400,0.44360,0.40810,0.03552,240000,1.609908e+09,1.609908e+09,2021-01-06 04:37:48,2.594595
84891,11599848.0,1609920708,weth2band2weth_sushiswap2Uniswap,I0016,4.726,0.05913,0.03249,0.02664,240000,1.609921e+09,1.609921e+09,2021-01-06 08:11:48,2.199917
88223,11602030.0,1609949107,weth2wbtc2weth_uniswap2Sushiswap,I0001,57.530,0.04677,0.01941,0.02736,240000,1.609949e+09,1.609949e+09,2021-01-06 16:05:07,2.273095
78949,11596263.0,1609872723,weth2inj2weth_uniswap2Sushiswap,I0019,3.442,0.04668,0.01404,0.03264,240000,1.609873e+09,1.609873e+09,2021-01-05 18:52:03,2.035936


## Statistics

In [9]:
# Limit opportuninities to the ones beyond the threshold to account for gas costs
net_profitable_opps = df[df.netProfit > 0]
opp_count = net_profitable_opps.blockNumber.count()
runtime_mean = net_profitable_opps.runTime.mean()
max_profit = net_profitable_opps.netProfit.max()
mean_profit = net_profitable_opps.netProfit.mean()
total_profit = net_profitable_opps.netProfit.sum()

In [10]:
print(f"Opportunitys count: {opp_count}")
print(f"Latency average: {runtime_mean:.2f} sec")
print(f"Max profit: {max_profit:.2f} ETH")
print(f"Average profit: {mean_profit:.2f} ETH")

Opportunitys count: 16
Latency average: 2.14 sec
Max profit: 0.41 ETH
Average profit: 0.06 ETH


## Instruction analysis

In [11]:
opp_comp_df = net_profitable_opps.groupby("instrName").agg({"blockNumber": "count", "netProfit": ["max", "sum", "mean"]})
opp_comp_df.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
opp_comp_df.index.name = None
opp_comp_df.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
weth2dai2weth_sushiswap2Uniswap,2,0.408100,0.816200,0.408100
weth2wbtc2weth_uniswap2Sushiswap,3,0.019410,0.042870,0.014290
weth2inj2weth_uniswap2Sushiswap,1,0.014040,0.014040,0.014040
weth2kp3rweth_sushiswap2Uniswap,1,0.012140,0.012140,0.012140
weth2band2weth_sushiswap2Uniswap,6,0.032490,0.038465,0.006411
weth2snx2weth_sushiswap2Uniswap,2,0.006209,0.012418,0.006209
weth2sushi2weth_uniswap2Sushiswap,1,0.000500,0.000500,0.000500


# Archer submissions

In [103]:
df_arch = pd.DataFrame(pd.read_csv("../logs/archer_api_requests.csv"))
df_arch.columns = [
                "bot_id", 
                "target_block", 
                "trade", 
                "estimated_profit_before_gas", 
                "gas_estimate", 
                "query", 
                "query_breakeven",
                "input_amount",
                "input_asset",
                "query_insert_locations", 
                "trade_insert_locations", 
                "response"
             ]

In [106]:
pd.set_option('display.max_colwidth', -1)
cut = df_arch[df_arch.target_block==11602794]
if len(cut):
    cut = cut.iloc[0]
print(cut)

bot_id                         2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

/home/mike/Desktop/mosquito/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


"0x6880608f00000000000000000000000000000000000000000000000000000000000000c0000000000000000000000000000000000000000000000000000000000000026000000000000000000000000000000000000000000000000000000000000002a0000000000000000000000000000000000000000000000000000000000000052000000000000000000000000000000000000000000000000029c9388c0ab7298000000000000000000000000000000000000000000000000029c9388c0ab380000000000000000000000000000000000000000000000000000000000000000170121835e15703a1a7bab32626d0927d60f90a81d700000000000000000000000000000000000000000000000000000000000000847f2b0c090000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d00000000000000000000000000000000000000000000000029c9388c0ab38000000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2121835e15703a1a7bab32626d0927d60f90a81d700000000000000000000000000000000000000000000000000000000000000847f2b0c09000000000000000000000000d9e1ce17f2641f24ae83637ab66a2cca9c378b9f00000000000000000000000000000000000000000000000fee1dead0fbadf00d0000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000011000000000000000000000000000000000000000000000000000000000000002507a250d5630b4cf539739df2c5dacb4c659f2488d00000000000000000000000000000000000000000000000000000000000000e47ff36ab500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000080000000000000000000000000d30ce37a6f2424593dabe9b712d235781815445d000000000000000000000000000000000000000000000000000000005ff4ff120000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2d9e1ce17f2641f24ae83637ab66a2cca9c378b9f000000000000000000000000000000000000000000000000000000000000010418cbafe500000000000000000000000000000000000000000000000fee1dead0fbadf00d000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000d30ce37a6f2424593dabe9b712d235781815445d000000000000000000000000000000000000000000000000000000005ff4ff1200000000000000000000000000000000000000000000000000000000000000020000000000000000000000006b3595068778dd592e39a122f4f5a5cf09c90fe2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc20000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000150"

SyntaxError: invalid syntax (<ipython-input-96-33f8603b2cc8>, line 1)